In [1]:
# Dependencies and Setup
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Rename columns to to replace " " with "_"
cols = purchase_data.columns
cols = cols.map(lambda x: x.replace(' ', '_') if isinstance(x, (str)) else x)
purchase_data.columns = cols
purchase_data.head()

,Purchase_ID,SN,Age,Gender,Item_ID,Item_Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
# Display the total number of players
total_players = purchase_data.SN.nunique()
total_players_dict = [{"Total Players" : total_players}]
total_players = pd.DataFrame(total_players_dict)
total_players

,Total Players
0,576


In [5]:
### Purchasing Analysis (Total) ###

# Run basic calculations to obtain number of unique items, average price, number of purchases and total revenue
unique_items = purchase_data.Item_ID.nunique()

# Format price columns
pd.options.display.float_format = "${:,.2f}".format
avg_price = purchase_data.Price.mean()

total_purchases = purchase_data.Purchase_ID.count()

total_revenue = purchase_data.Price.sum()

basic_calc_dict = [{"Number of Unique Items" : unique_items, 
                   "Average Price" : avg_price, 
                   "Number of Purchases" : total_purchases, 
                   "Total Revenue" : total_revenue}]

basic_calc = pd.DataFrame(basic_calc_dict)

basic_calc

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [6]:
### Gender Demographics ###

# Filter data by SN for unique entires
unique_SN = purchase_data.drop_duplicates('SN')

unique_gender = unique_SN.Gender.value_counts()
unique_gender = pd.DataFrame(unique_gender)

total_gender = unique_SN.Gender.count()
unique_gender

# Percentage and Count of Male, Female and  Players
pd.options.display.float_format = "{:,.2f}".format
gender_percent = (unique_gender/total_gender)*100
unique_gender["Percentage of Players"] = gender_percent
unique_gender.rename(columns = {'Gender':'Total Count'}, inplace = True)

unique_gender

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [7]:
### Purchasing Analysis (Gender) ###

# purchase count
gender_count = purchase_data.Gender.value_counts()
gender_count = pd.DataFrame(gender_count)
gender_count.sort_index()
gender_count.index.name = "Gender"
gender_count.rename(columns = {'Gender':'Purchase Count'}, inplace = True)

# total purchase price
total_purch = purchase_data.groupby(['Gender'])['Price'].sum()
gender_count['Total Purchase Value'] = pd.Series(total_purch)

# avg. purchase price
avg_purch_price = gender_count['Total Purchase Value']/gender_count['Purchase Count']
gender_count['Average Purchase Price'] = pd.Series(avg_purch_price)

# avg. total purchase per person = sum(female purchases) / sum(unique females)
avg_total_pp = gender_count['Total Purchase Value']/unique_gender['Total Count']
gender_count['Avg Total Purchase per Person'] = pd.Series(avg_total_pp)
gender_count

# re-order columns
gender_count = gender_count[['Purchase Count', 'Average Purchase Price', 
                             'Total Purchase Value', 'Avg Total Purchase per Person']]

# set column formatting
format_mapping = {'Purchase Count': '{:,.0f}', 'Average Purchase Price': '${:,.2f}', 
                'Total Purchase Value': '${:,.2f}', 'Avg Total Purchase per Person': '${:,.2f}'}

for key, value in format_mapping.items():
    gender_count[key] = gender_count[key].apply(value.format)
    
# sort index
gender_count = gender_count.sort_index()
    
gender_count

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [8]:
### Age Demographics ###

# Filter data by SN for unique entires
age_unique_SN = purchase_data.drop_duplicates('SN')

# Establish bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_unique_SN["Total Count"] = pd.cut(age_unique_SN["Age"], bins, labels=group_names)

# Create new dataframe
age_demo = age_unique_SN["Total Count"].value_counts()
age_demo = pd.DataFrame(age_demo)

age_demo = age_demo.sort_index()

# Find percentages
age_demo_sum = age_demo["Total Count"].sum()

age_demo_perc = (age_demo["Total Count"]/age_demo_sum)*100
age_demo['Percentage of Players'] = pd.Series(age_demo_perc)

age_demo

,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [9]:
### Purchasing Analysis (Age) ###

# Create new dataframe
purch = pd.DataFrame(purchase_data)

# Add column with bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purch["Age Bins"] = pd.cut(purch["Age"], bins, labels=group_names)

# Count bins and set index as age bins, rename first column
age_bin_count = purch["Age Bins"].value_counts()
age_bin_count = pd.DataFrame(age_bin_count)
age_bin_count.sort_index()
age_bin_count.rename(columns = {'Age Bins':'Purchase Count'}, inplace = True)

# Total Purchase Value
total_purch = purch.groupby(['Age Bins'])['Price'].sum()
age_bin_count['Total Purchase Value'] = pd.Series(total_purch)

# Avg. Purchase Price
avg_purch_price = age_bin_count['Total Purchase Value']/age_bin_count['Purchase Count']
age_bin_count['Average Purchase Price'] = pd.Series(avg_purch_price)

# avg. total purchase per person = sum(female purchases) / sum(unique females)
avg_total_pp = age_bin_count['Total Purchase Value']/age_demo['Total Count']
age_bin_count['Avg Total Purchase per Person'] = pd.Series(avg_total_pp)

# re-order columns
age_bin_count = age_bin_count[['Purchase Count', 'Average Purchase Price', 
                             'Total Purchase Value', 'Avg Total Purchase per Person']]

# set column formatting
format_mapping = {'Purchase Count': '{:,.0f}', 'Average Purchase Price': '${:,.2f}', 
                'Total Purchase Value': '${:,.2f}', 'Avg Total Purchase per Person': '${:,.2f}'}

for key, value in format_mapping.items():
    age_bin_count[key] = age_bin_count[key].apply(value.format)
    
# sort index
age_bin_count = age_bin_count.sort_index()
    
age_bin_count

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [10]:
### Top Spenders ###

# Create dataframe with SN index and summing on price to get total value
top_spend = purchase_data.groupby(['SN'])['Price'].sum()
top_spend = pd.DataFrame(top_spend)
top_spend.rename(columns = {'Price':'Total Purchase Value'}, inplace = True)

# Value counts on SN to get purchase count
purch_count = purchase_data.SN.value_counts()
top_spend['Purchase Count'] = pd.Series(purch_count)
top_spend['Average Purchase Price'] = top_spend['Total Purchase Value'] / top_spend['Purchase Count']

# Re-order columns formatting and sort
top_spend = top_spend[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']]

top_spend = top_spend.sort_values(by=['Total Purchase Value'], ascending=False)

format_mapping = {'Purchase Count': '{:,.0f}', 'Average Purchase Price': '${:,.2f}', 
                'Total Purchase Value': '${:,.2f}'}

for key, value in format_mapping.items():
    top_spend[key] = top_spend[key].apply(value.format)

top_spend.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [11]:
purchase_data.head()

,Purchase_ID,SN,Age,Gender,Item_ID,Item_Name,Price,Age Bins
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [33]:
### Most Popular Items ###

popular = purchase_data[['Item_ID', 'Item_Name', 'Price']]
popular.rename(columns = {'Price':'Item Price'}, inplace = True)
popular.head()

# popular['Total Purchase Value'] = popular['Item Price'] * popular['Purchase Count']

# popular = popular.sort_values(by=['Purchase Count'], ascending=False)



,Item_ID,Item_Name,Item Price
0,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,143,Frenzied Scimitar,1.56
2,92,Final Critic,4.88
3,100,Blindscythe,3.27
4,131,Fury,1.44


In [29]:
### Most Profitable Items ###

# Sort the above table by total purchase value in descending order


# Optional: give the displayed data cleaner formatting


# Display a preview of the data frame
